In [1]:
import sys, struct
import io,PIL
from PIL import Image

In [38]:
filePath="/data/Rat106/Experiments/Rat106_2017_03_31_10_56/Rat106_2017_03_31_10_56.seq"

In [52]:
try:f.close()
except:pass
f=open(filePath,'br')

In [55]:
f.seek(572)
data=f.read(4)
print(struct.unpack('=L',data))
f.seek(-576,1)

(293360,)


0

In [26]:
f.seek(1024)
frameSizeByte=f.read(4)

In [27]:
frameSize=int.from_bytes(frameSizeByte,sys.byteorder)-4

In [28]:
frameMs=f.read(frameSize)

In [7]:
image = Image.open(io.BytesIO(frameMs))
image.show()

In [29]:
t0=f.read(4)
t1=f.read(2)
t2=f.read(2)

In [30]:
print(struct.unpack('=l',t0))
print(struct.unpack('=H',t1))
print(struct.unpack('=H',t2))

(1490950590,)
(729,)
(842,)


In [33]:
f.seek(-8,1)
t=f.read(8)
print(struct.unpack('=lHH',t))

(1490950590, 729, 842)


In [ ]:
import io,inspect
inspect.signature(io.BufferedIOBase.__enter__)

In [60]:
class SeqFile():
    def __init__(self, *arg, **kw):
        self.arg=arg
        self.kw=kw
        
    def __enter__(self):
        try:
            self.file_obj = open(*self.arg, **self.kw)
        except:
            logging.error(f"file: {self.arg} failed to open!")
            return None
        
        return self.file_obj

    def __exit__(self, *args):
        
        self.file_obj.close()
    
    

AttributeError: type object 'SeqFile' has no attribute 'open'